In [3]:
import pandas as pd


In [4]:
ds=pd.read_csv(r'marketing_sample_for_naukri_com-jobs__20190701_20190830__30k_data.csv')
ds.head(1)

,Uniq Id,Crawl Timestamp,Job Title,Job Salary,Job Experience Required,Key Skills,Role Category,Location,Functional Area,Industry,Role
0,9be62c49a0b7ebe982a4af1edaa7bc5f,2019-07-05 01:46:07 +0000,Digital Media Planner,Not Disclosed by Recruiter,5 - 10 yrs,Media Planning| Digital Media,Advertising,Mumbai,"Marketing , Advertising , MR , PR , Media Plan...","Advertising, PR, MR, Event Management",Media Planning Executive/Manager


In [9]:
ds.columns

Index(['Uniq Id', 'Crawl Timestamp', 'Job Title', 'Job Salary',
       'Job Experience Required', 'Key Skills', 'Role Category', 'Location',
       'Functional Area', 'Industry', 'Role'],
      dtype='object')

In [10]:
ds.drop(['Uniq Id','Crawl Timestamp'],axis=1,inplace=True)

In [11]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Job Title                29425 non-null  object
 1   Job Salary               29950 non-null  object
 2   Job Experience Required  29427 non-null  object
 3   Key Skills               28729 non-null  object
 4   Role Category            27695 non-null  object
 5   Location                 29423 non-null  object
 6   Functional Area          29427 non-null  object
 7   Industry                 29427 non-null  object
 8   Role                     29099 non-null  object
dtypes: object(9)
memory usage: 2.1+ MB


In [12]:
ds['Key Skills'][29998]

' Security Analysis| Software Development Life Cycle| Security Testing| Cyber Security| Application Security| New Projects| Code Review| Project Development| Software Solutions| Security Services'

In [13]:
ds.shape

(30000, 9)

In [14]:
ds['Skills']=ds['Key Skills'].str.split("|")

In [15]:
ds.head(1)

,Job Title,Job Salary,Job Experience Required,Key Skills,Role Category,Location,Functional Area,Industry,Role,Skills
0,Digital Media Planner,Not Disclosed by Recruiter,5 - 10 yrs,Media Planning| Digital Media,Advertising,Mumbai,"Marketing , Advertising , MR , PR , Media Plan...","Advertising, PR, MR, Event Management",Media Planning Executive/Manager,"[ Media Planning, Digital Media]"


In [16]:
ds.columns

Index(['Job Title', 'Job Salary', 'Job Experience Required', 'Key Skills',
       'Role Category', 'Location', 'Functional Area', 'Industry', 'Role',
       'Skills'],
      dtype='object')

In [17]:
ds.isnull().sum()

Job Title                   575
Job Salary                   50
Job Experience Required     573
Key Skills                 1271
Role Category              2305
Location                    577
Functional Area             573
Industry                    573
Role                        901
Skills                     1271
dtype: int64

In [18]:
ds.isnull().sum()

Job Title                   575
Job Salary                   50
Job Experience Required     573
Key Skills                 1271
Role Category              2305
Location                    577
Functional Area             573
Industry                    573
Role                        901
Skills                     1271
dtype: int64

In [19]:
ds.columns

Index(['Job Title', 'Job Salary', 'Job Experience Required', 'Key Skills',
       'Role Category', 'Location', 'Functional Area', 'Industry', 'Role',
       'Skills'],
      dtype='object')

In [20]:
ds.dropna(subset=['Job Title', 'Industry','Skills','Role'], inplace=True)  # Drops rows where these columns have NaN


In [21]:
ds.columns

Index(['Job Title', 'Job Salary', 'Job Experience Required', 'Key Skills',
       'Role Category', 'Location', 'Functional Area', 'Industry', 'Role',
       'Skills'],
      dtype='object')

In [22]:
ds.isnull().sum()

Job Title                     0
Job Salary                    0
Job Experience Required       0
Key Skills                    0
Role Category              1362
Location                      0
Functional Area               0
Industry                      0
Role                          0
Skills                        0
dtype: int64

In [5]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [28]:
ds.to_csv(r'C:\Users\Asus\Contacts\Desktop\data sci projects\career path recommendation\processed_naukri.csv')

In [6]:
mod=pd.read_csv(r'processed_naukri.csv')

In [31]:
mod.shape

(28367, 12)

In [8]:
def clean_skills(skills):
    """
    Cleans and standardizes skills format whether input is a string, list, or set.
    """
    cleaned_skills = set()
    
    try:
        # Handle different input types
        if isinstance(skills, str):
            # If it's a string that looks like a list
            if skills.strip().startswith('[') and skills.strip().endswith(']'):
                try:
                    # Try to safely evaluate the string as a list
                    skills_list = eval(skills)
                    if isinstance(skills_list, list):
                        skills = skills_list
                    else:
                        skills = skills.strip('[]').split(',')
                except:
                    skills = skills.strip('[]').split(',')
            else:
                # Normal comma-separated string
                skills = skills.split(',')
        
        # Convert to list if it's a set
        if isinstance(skills, set):
            skills = list(skills)
            
        # Ensure we're working with a list
        if not isinstance(skills, list):
            skills = [str(skills)]
        
        # Clean each skill
        for skill in skills:
            if isinstance(skill, (str, int, float)):
                cleaned_skill = str(skill).strip().strip("'").strip('"').lower()
                if cleaned_skill:
                    cleaned_skills.add(cleaned_skill)
                    
        return cleaned_skills
        
    except Exception as e:
        print(f"Error in cleaning skills: {str(e)}")
        return set()


In [28]:
def calculate_skill_gap(job_skills, user_skills):
    """
    Calculates missing skills using fuzzy matching and word embeddings principles.
    
    Args:
        job_skills (set): Required skills for the job
        user_skills (set): User's current skills
    
    Returns:
        set: Missing skills that the user needs
    """
    missing_skills = set()
    matched_skills = set()
    
    # Convert sets to lowercase for comparison
    job_skills = {skill.lower() for skill in job_skills}
    user_skills = {skill.lower() for skill in user_skills}
    
    for job_skill in job_skills:
        skill_found = False
        
        # Check for exact matches first
        if job_skill in user_skills:
            matched_skills.add(job_skill)
            skill_found = True
            continue
            
        # Check for substring matches
        for user_skill in user_skills:
            # Check both directions of substring containment
            if (job_skill in user_skill) or (user_skill in job_skill):
                matched_skills.add(job_skill)
                skill_found = True
                break
                
            # Check for common words in multi-word skills
            job_words = set(job_skill.split())
            user_words = set(user_skill.split())
            common_words = job_words.intersection(user_words)
            
            # If significant word overlap, consider it a match
            if len(common_words) / max(len(job_words), len(user_words)) > 0.5:
                matched_skills.add(job_skill)
                skill_found = True
                break
        
        if not skill_found:
            missing_skills.add(job_skill)
    
    return missing_skills

def recommend_jobs(user_skills, top_n=10):
    
    try:
        # Clean user skills
        user_skills_set = clean_skills(user_skills)
        if not user_skills_set:
            raise ValueError("No valid user skills found after cleaning")
            
        # Create a copy of the dataframe
        mod_copy = mod.copy()
        
        # Clean skills in the dataset
        mod_copy["Clean_Skills"] = mod_copy["Skills"].apply(clean_skills)
        
        # Prepare skills for vectorization
        skills_list = [" ".join(sorted(list(skills))) for skills in mod_copy["Clean_Skills"]]
        skills_list.append(" ".join(sorted(list(user_skills_set))))
        
        # TF-IDF vectorization
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(skills_list)
        
        # Calculate similarity
        similarity_scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
        
        # Get top matches
        top_indices = similarity_scores.argsort()[-top_n:][::-1]
        
        top_jobs = []
        for idx in top_indices:
            job_skills = mod_copy.iloc[idx]["Clean_Skills"]
            
            # Calculate missing skills using the new function
            missing_skills = calculate_skill_gap(job_skills, user_skills_set)
            
            # Calculate match percentage
            total_required = len(job_skills)
            if total_required > 0:
                missing_count = len(missing_skills)
                match_percentage = round(((total_required - missing_count) / total_required) * 100, 1)
            else:
                match_percentage = 0.0
            
            job_info = {
                "Job Title": mod_copy.iloc[idx]["Job Title"],
                "Role": mod_copy.iloc[idx]["Role"],
                "Industry": mod_copy.iloc[idx]["Industry"],
                "Required Skills": sorted(list(job_skills)),
                "Missing Skills": sorted(list(missing_skills)),
                "Match Percentage": match_percentage,
                "Similarity Score": round(similarity_scores[idx], 3)
            }
            top_jobs.append(job_info)
        
        return top_jobs
    
    except Exception as e:
        print(f"Error in job recommendation: {str(e)}")
        return []

In [27]:
def format_recommendations(recommendations):
  
    if not recommendations:
        return "No matching jobs found."
        
    output = []
    for i, job in enumerate(recommendations, 1):
        output.extend([
            f"\n{i}. Job Details:",
            f"   Title           : {job['Job Title']}",
            f"   Role            : {job['Role']}",
            f"   Industry        : {job['Industry']}",
            f"   Match           : {job['Match Percentage']}%",
            f"   Required Skills : {', '.join(job['Required Skills'])}",
            f"   Missing Skills  : {', '.join(job['Missing Skills']) if job['Missing Skills'] else 'None'}",
            "-" * 80
        ])

    
    
    return "\n".join(output)



In [15]:
user_input = "Python, SQL, Machine Learning, Data Analysis, Statistics"

recommendations = recommend_jobs(user_input, top_n=5)
print(format_recommendations(recommendations))


1. Job Details:
   Title           :  Business Analyst - SQL & R - Retail/ecommerce/telecom
   Role            : Business Analyst
   Industry        : Internet, Ecommerce
   Match           : 50.0%
   Required Skills : analytics, data analytics, data management, machine learning, sql, statistics
   Missing Skills  : analytics, data analytics, data management
--------------------------------------------------------------------------------

2. Job Details:
   Title           :  Trainee-data Scientist
   Role            : Trainee
   Industry        : IT-Software, Software Services
   Match           : 100.0%
   Required Skills : machine learning, python
   Missing Skills  : None
--------------------------------------------------------------------------------

3. Job Details:
   Title           :  Data Scientist
   Role            : Bio-Statistician
   Industry        : Banking, Financial Services, Broking
   Match           : 44.4%
   Required Skills : big data, data quality, data scienc

In [16]:
courses=pd.read_csv(r'coursea_data.csv')


In [17]:
courses.shape

(891, 7)

In [20]:
courses.columns

Index(['Unnamed: 0', 'course_title', 'course_organization',
       'course_Certificate_type', 'course_rating', 'course_difficulty',
       'course_students_enrolled'],
      dtype='object')

In [23]:
courses.head()

,Unnamed: 0,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled
0,134,(ISC)² Systems Security Certified Practitioner...,(ISC)²,SPECIALIZATION,4.7,Beginner,5.3k
1,743,A Crash Course in Causality: Inferring Causal...,University of Pennsylvania,COURSE,4.7,Intermediate,17k
2,874,A Crash Course in Data Science,Johns Hopkins University,COURSE,4.5,Mixed,130k
3,413,A Law Student's Toolkit,Yale University,COURSE,4.7,Mixed,91k
4,635,A Life of Happiness and Fulfillment,Indian School of Business,COURSE,4.8,Mixed,320k


In [29]:


def recommend_courses(user_skills, df, num_recommendations=5):
 
   
    # Clean and prepare the input
    user_skills = user_skills.lower().strip()
    
    # Create TF-IDF vectorizer
    tfidf = TfidfVectorizer(stop_words='english')
    
    # Fit TF-IDF on course titles
    course_title_matrix = tfidf.fit_transform(df['course_title'].str.lower())
    
    # Transform user skills
    user_skills_matrix = tfidf.transform([user_skills])
    
    # Calculate similarity scores
    similarity_scores = cosine_similarity(user_skills_matrix, course_title_matrix)
    
    # Get indices of top recommendations
    course_indices = similarity_scores[0].argsort()[-num_recommendations:][::-1]
    
    # Get recommended courses with their details
    recommended_courses = df.iloc[course_indices].copy()
    
    # Add similarity score to the recommendations
    recommended_courses['similarity_score'] = similarity_scores[0][course_indices]
    
    # Sort by both similarity score and rating
    recommended_courses['combined_score'] = (
        0.7 * recommended_courses['similarity_score'] + 
        0.3 * recommended_courses['course_rating']
    )
    recommended_courses = recommended_courses.sort_values('combined_score', ascending=False)
    
    # Format the output
    output_columns = [
        'course_title', 'course_organization', 'course_Certificate_type',
        'course_rating', 'course_difficulty', 'course_students_enrolled'
    ]
    
    return recommended_courses[output_columns]



In [26]:
user_skills="Python, Sql"
recommend_courses(user_skills,courses, num_recommendations=5)

,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled
711,SQL for Data Science,"University of California, Davis",COURSE,4.6,Beginner,160k
208,Databases and SQL for Data Science,IBM,COURSE,4.7,Beginner,110k
610,Modern Big Data Analysis with SQL,Cloudera,SPECIALIZATION,4.8,Beginner,14k
553,Learn SQL Basics for Data Science,"University of California, Davis",SPECIALIZATION,4.6,Beginner,160k
752,Statistics with Python,University of Michigan,SPECIALIZATION,4.6,Beginner,34k


In [ ]:
#call course recommendation function for all the missing skills
